## Add TIRCP

In [1]:
import _utils
import pandas as pd
import numpy as np
from calitp.sql import to_snakecase

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
# import fuzzywuzzy
# from fuzzywuzzy import process

### ATP - 10 Year Non SHOPP Prep

In [4]:
# Read in 10 Year non SHOPP
atp_shopp = to_snakecase(
    pd.read_excel(f"{_utils.GCS_FILE_PATH}cleaned_data_with_fake_metrics_plus_atp.xlsx")
)

In [5]:
atp_shopp.head(1)

,unnamed:_0,rail_project_id,_10_year_plan,ct_project_id,update_complete,ea,ppno,project_name,district,county,route,project_description,current_phase,con_existing_source_s__of_funds,con_anticipated_source_of_funds,target_opening_year,beg_pm,end_pm,primary_mode,previous_caltrans_nominations,urban_rural,notes,lead_agency,pid_approval_date__m010,target_pa_ed__m200,rtl_date__m460,con_start_date__m500,funding_need_phase_s,"pa_ed_cost__$1,000","ps_e_cost__$1,000","row_cost__$1,000","con_support_cost__$1,000","non_infrastructure___plan_cost__$1,000","total_unfunded_need__$1,000",previous_funding_request_phase,last_scored,csis_alignment,csis_total_score__out_of_45,mode_shift__csis__score,mode_shift__csis__comment,vmt__csis__score,vmt__csis__comment,public_engagement__csis__score,public_engagement__csis__comment,dac_local_community_needs__csis__score,dac_local_community_needs__csis__comment,safety__csis__score,safety__csis__comment,zev__csis__score,zev__csis__comment,climate_resiliency__csis__score,climate_resiliency__csis__comment,natural_resources_and_ecosystems__csis__score,natural_resources_and_ecosystems__csis__comment,infill_development_and_land_use__csis__score,infill_development_and_land_use__csis__comment,benefits_to_dac_and_advancing_equity__atp__score,community_need__atp__score,safety__atp__score,public_participation__atp__score,community_feedback__atp__score,continued_engagement__atp__score,context_sensitive_and_innovation__atp__score,transformative__atp__score,atp_total_score__out_of_100,atp_alignment,access_alignment,_2023,_2024,_2025,_2026,_2027,_2028,_2029,_2030,_2031,_2032,_2033,previous_funding_request,purpose___need,parcel_counts,"total_project_cost__$1,000","con_capital_cost__$1,000",hq_priority,district_priority,potential_funding_program_s,located_in_dac,shs_capacity_increase_detail,secondary_mode_s,full_county_name,abbrev,merge,detailed_project_title,district_full_name,ppno1,"total_project_cost__$1,000_1","pa_ed_cost__$1,000_1","ps_e_cost__$1,000_1","non_infrastructure___plan_cost__$1,000_1"
0,0,None,0.0,0118000113,1.0,0H830,2503,Koster Couplet,1,HUM,NaN,"US 101 Couplet for Safety, Livability, and Sustainability. Add three lanes of travel for vehicle operations with two lanes southbound using Koster Street and one additional lane northbound for one alternative. Traffic Calming features include adding Class II bike lanes, widen sidewalks, add parking, and other complete streets features.",REMOVE/END,Information currently not available,Information currently not available,2030,77.2,78.113,Complete Streets,SHOPP,Rural,PID Completed,Caltrans,datetime64[ns],2023-04-03 00:00:00,datetime64[ns],datetime64[ns],0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,Not Well-Aligned,0.0,0.0,None,0.0,None,0.0,None,0.0,0.0,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not Well-Aligned,None,None,None,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,0,62500.0,0.0,0.0,0.0,Atp\nActive Transportation Infrastructure Investment Program\nRaise,None,General Purpose Lane,None,Humboldt,HUM,both,District 1-Koster Couplet,01 - Eureka,NaN,NaN,NaN,NaN,NaN


In [6]:
# Subset
atp_shopp_subset = [
    "ppno",
    "project_name",
    "lead_agency",
    "previous_caltrans_nominations",
    "full_county_name",
    "district",
    "project_description",
    "current_phase",
    "primary_mode",
    "urban_rural",
    'total_project_cost__$1,000',
    "total_unfunded_need__$1,000",
    "notes",
    'shs_capacity_increase_detail'
]

In [7]:
atp_shopp2 = atp_shopp[atp_shopp_subset]

In [8]:
# atp_shopp2.sample()

In [9]:
# Function to clean agency/organization names
def organization_cleaning(df, column_wanted: str):
    df[column_wanted] = (
        df[column_wanted]
        .str.strip()
        .str.split(",")
        .str[0]
        .str.replace("/", "")
        .str.split("(")
        .str[0]
        .str.split("/")
        .str[0]
        .str.title()
        .str.replace("Trasit", "Transit")
        .str.strip()  # strip again after getting rid of certain things
    )
    return df

In [10]:
# Lowercase previous caltrans nominations
atp_shopp2.previous_caltrans_nominations = (
    atp_shopp2.previous_caltrans_nominations.str.lower()
)

/opt/conda/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
# Clean ATP 
atp_shopp2 = organization_cleaning(atp_shopp2, "lead_agency")

/tmp/ipykernel_1929/2101408362.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
# atp_shopp2.head(100)

### TIRCP Prep
* Filter out projects in which total cost equals total award 

In [13]:
# Read in TIRCP. Last updated November 2022.
tircp = to_snakecase(
    pd.read_excel(
        "gs://calitp-analytics-data/data-analyses/tircp/Tableau_Workbook.xlsx"
    )
)

In [14]:
# tircp.columns

In [15]:
# Subset TIRCP with only the basic information
tircp_subset = [
    "award_year",
    "grant_recipient",
    "title",
    "ppno",
    "district",
    "county",
    "description",
    "total__cost",
    "tircp",
    "award_cycle",
    "on_shs?",
    "comments_additional_contacts"
]

In [16]:
tircp2 = tircp[tircp_subset]

In [17]:
# Create a column with cycle + tircp for previous CT nominations
tircp2["previous_caltrans_nominations"] = "TIRCP" + " Cycle " + tircp2["award_cycle"].astype("str")

/tmp/ipykernel_1929/3217889362.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
# Create column for unmet needs
tircp2["total_unfunded_need__$1,000"] = tircp2["total__cost"] - tircp2["tircp"]

/tmp/ipykernel_1929/2212343423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [19]:
# Perhaps to narrow down projects
# Figure out which TIRCP projects' total cost are completely covered by TIRCP requested
tircp2["total_cost_vs_tircp_req"] =  tircp2["tircp"]/tircp2["total__cost"]

/tmp/ipykernel_1929/2443383145.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [20]:
# tircp2.total_cost_vs_tircp_req.value_counts()

In [21]:
# Filter out projects in which total cost equals total award 
tircp2 = (tircp2.loc[tircp2.total_cost_vs_tircp_req != 1]).reset_index(drop=True)

In [22]:
# Drop some columns
tircp_cols_drop = ['tircp', 'total_cost_vs_tircp_req', 'award_cycle']

In [23]:
tircp2 = tircp2.drop(columns = tircp_cols_drop)

In [24]:
tircp2.shape

(93, 12)

### Find TIRCP projects that are already in the 10 year Non SHOPP. 

#### Harmonize the way columns are named/formatted.
* TIRCP: `Total_cost` and `unmet needs` should be floats and divided by 1000. `District` should be int64. 

In [25]:
atp_shopp2.previous_caltrans_nominations.nunique(), len(atp_shopp2)

(242, 814)

In [26]:
# Find projects that have DRMT in previous caltrans nominations
atp_shopp_drmt = (
    atp_shopp2[atp_shopp2["previous_caltrans_nominations"].str.contains(("tircp|drmt"))]
).reset_index(drop=True)

In [27]:
# Check out that the filtering was correct. 
# atp_shopp_drmt.previous_caltrans_nominations.value_counts()

In [28]:
# Natalie's function
def align_funding_numbers(df, list_of_cols):
    for col in list_of_cols:
        df[col] = df[col]/1000
        
    return df

In [29]:
tircp2 = align_funding_numbers(tircp2, ["total_unfunded_need__$1,000", "total__cost",])

In [30]:
# (tircp2[["total_unfunded_need__$1,000"]]/1000).astype('int64')*1000

In [31]:
tircp2[["total_unfunded_need__$1,000", "total__cost",]] = tircp2[["total_unfunded_need__$1,000", "total__cost",]].fillna(0)

In [32]:
tircp2["district_full_name"] = tircp2["district"]

In [33]:
# Extract digit from district. Fill "Various" districts with "0" and change to int64
tircp2["district"] = tircp2["district"].str.extract('(\d+)').fillna(0).astype('int64')

In [34]:
# Rename columns
tircp_new_cols = {'grant_recipient': 'lead_agency',
                  'title': 'project_name',
                  'county': 'full_county_name', 
                  'description': 'project_description',
                  'total__cost': 'total_project_cost__$1,000', 
                  'on_shs?': 'shs_capacity_increase_detail',
                  'comments_additional_contacts': 'notes',
                 }

In [35]:
# atp_shopp_drmt.info(), tircp2.info()

In [36]:
tircp2 = tircp2.rename(columns = tircp_new_cols)

#### Test with Merges
* Zero merges across the board. 

In [37]:
# Make a copy of tircp
# tircp_test = tircp2.copy()

In [38]:
# Round numbers
# tircp_test["total_unfunded_need__$1,000"] = (tircp_test[["total_unfunded_need__$1,000"]]/1000).astype('int64')*1000

In [39]:
# tircp_test["total_project_cost__$1,000"] = (tircp_test[["total_project_cost__$1,000"]]/1000).astype('int64')*1000

In [40]:
# atp_shopp_drmt["total_project_cost__$1,000"] = ((atp_shopp_drmt[["total_project_cost__$1,000"]]/1000).astype('int64')*1000).astype('int64')
# atp_shopp_drmt["total_project_cost__$1,000"] = ((atp_shopp_drmt[["total_project_cost__$1,000"]]/1000).astype('int64')*1000).astype('int64')

In [41]:
# Merge on district and unfunded needs
# test1 = pd.merge(atp_shopp_drmt, tircp2, how= "outer", on=["district", "total_unfunded_need__$1,000"], indicator = True)

In [42]:
# test1._merge.value_counts()

In [43]:
# Merge on county and unfunded needs
# test2 = pd.merge(atp_shopp_drmt, tircp2, how= "left", on=["full_county_name", "total_unfunded_need__$1,000"], indicator = True, suffixes = ["_shopp", "_tircp"])

In [44]:
# test2._merge.value_counts()

In [45]:
# test2.loc[test2._merge == "both"][["project_name_shopp","project_name_tircp", "full_county_name", "project_description_shopp","project_description_tircp"]]

In [46]:
# Merge on district and total costs 
# test2 = pd.merge(atp_shopp_drmt, tircp2, how= "outer", on=["district", "total_project_cost__$1,000"], indicator = True)

#### Observation: 
* Total Project Cost, project titles, lead agency, and districts are potentially listed differently across datasets.
* Inglewood Transit Connector Project	  is listed as $1,016,000.000 in TIRCP but 1,666,466.0 in non SHOPP.
    * City Of Inglewood is the grant recipient in TIRCP but Caltrans is the lead agency in Non SHOPP.
* Valley Rail Expansion: Altamont Corridor Express (ACE) Ceres to Turlock Extension (the TIRCP title) is listed as D6 in TIRCP but D10 in Stanislaus.
    * Fresno Subdivision (Ceres To Turlock) Double Tracking is the non SHOPP title.

In [47]:
tircp_already_entered = ['Inglewood Transit Connector Project','Valley Rail Expansion: Altamont Corridor Express (ACE) Ceres to Turlock Extension',]

In [48]:
# Projects with only TIRCP in the previous nomination.
len(atp_shopp_drmt[atp_shopp_drmt["previous_caltrans_nominations"].str.contains(("tircp"))])

3

In [49]:
# Delete TIRCP projects that are already in non SHOPP
# One project below in non SHOPP mentions TIRCP but that project
# was ultimately not nominated.
tircp2 = (tircp2[~tircp2["project_name"].isin(tircp_already_entered)]).reset_index(drop = True)

In [50]:
# atp_shopp_drmt[atp_shopp_drmt["previous_caltrans_nominations"].str.contains(("tircp"))].drop(columns = "notes")

#### Test with Project Names

In [51]:
tircp2["project_test"] = tircp2["project_name"]

In [52]:
atp_shopp_drmt["project_test"] = atp_shopp_drmt["project_name"]

In [53]:
def simplify_project_names(df, column_wanted: str):
    df[column_wanted] = (
        df[column_wanted]
        .str.strip()
        .str.lower()
        .str.replace("/", "")
        .str.replace("-","")
        .str.replace("!","")
        .str.replace("&","")
        .str.replace("#","")
        .str.replace("(", "")
        .str.replace(")", "")
        .str.replace(":","")
        .str.replace("the","")
        .str.strip() # strip again after getting rid of certain things
    )
    return df

In [54]:
tircp2 = simplify_project_names(tircp2, "project_test")

/tmp/ipykernel_1929/1193460345.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [55]:
atp_shopp_drmt = simplify_project_names(atp_shopp_drmt, "project_test")

/tmp/ipykernel_1929/1193460345.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [56]:
# atp_shopp_drmt[["project_name","project_test"]].sort_values(by = ["project_name"])

In [57]:
# Merge on project names
test3 = pd.merge(atp_shopp_drmt, tircp2, how= "outer", on=["project_test"], indicator = True, suffixes = ["_shopp", "_tircp"] )

In [58]:
test3._merge.value_counts()

left_only     167
right_only     91
both            0
Name: _merge, dtype: int64

In [59]:
# atp_shopp_drmt[["project_test"]].sort_values("project_test")

In [60]:
# tircp2[["project_test"]].sort_values("project_test")

#### Test 1 with Fuzzy Matching on Project Names
* Unsuccessful with project names. Only 2 matches and after looking at descriptions/districts manually, they aren't similar enough.

In [61]:
# Replace all rows in agency column with a min ratio with  "string_to_match value"
def replace_matches_in_column(df, column, new_col_name, string_to_match, min_ratio):
    # Get a list of unique strings
    strings = df[column].unique()

    # Get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(
        string_to_match, strings, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio
    )

    # Only get matches with a  min ratio
    close_matches = [matches[0] for matches in matches if matches[1] > min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches
    df.loc[rows_with_matches, new_col_name] = string_to_match

In [62]:
# Create a list of all the TIRCP names 
# tircp_projects = tircp2.project_name.unique().tolist()

In [63]:
# atp_shopp_projects = atp_shopp_drmt.project_name.unique().tolist()

In [64]:
#for i in atp_shopp_projects:
#    replace_matches_in_column(
#        tircp2, "project_name", "project_name_fuzzy_match", i, 80
#    )

In [65]:
# tircp2[["project_description", "project_description","project_description_fuzzy_match"]]

In [66]:
# Merge on project names
# test4 = pd.merge(atp_shopp_drmt, tircp2, how= "outer", left_on=["project_test"], right_on = ["project_name_fuzzy_match"], indicator = True, suffixes = ["_shopp", "_tircp"] )

In [67]:
# test4._merge.value_counts()

In [68]:
# test4.loc[test4._merge == "both"][["district_shopp","district_tircp","project_name_tircp","project_name_shopp", "project_description_shopp", "project_description_tircp"]]

### Concat
* After only finding 2 projects that are already entered in, concat TIRCP information.

In [69]:
# Drop columns
tircp2 = tircp2.drop(columns = ['project_test', 'award_year'])

In [70]:
# Add column for primary mode. 
tircp2["primary_mode"] = "Transit/Zev/Rail (Passenger)" 

In [71]:
atp_drop = ['unnamed:_0','update_complete'] 

In [72]:
atp_shopp = atp_shopp.drop(columns = atp_drop)

In [73]:
concat1 = pd.concat([atp_shopp, tircp2])

In [74]:
# Fill NA based on dtypes
concat1 = concat1.fillna(
    concat1.dtypes.replace({"float64": 0.0, "object": "None"})
)

In [75]:
# concat1.info(verbose=True)

In [76]:
# concat1.tail(1)

### Add Fake Values

In [77]:
# https://stackoverflow.com/questions/64093880/how-to-create-random-floats-and-add-them-as-a-dataframe-column
np.random.seed(365)

In [78]:
fake_columns = [
    "increase_peak_person_throughput",
    "reduction_in_peak_period_delay",
    "reduction_in_fatal_and_injury_crashes",
    "reduction_in_injury_rates",
    "increase_access_to_jobs",
    "increase_access_jobs_to_DAC",
    "commercial_dev_developed",
    "tons_of_goods_impacted",
    "improve_air_quality",
    "impact_natural_resources",
    "support_of_transportation",
]

In [79]:
# Add fake metric columns: I want a random value between 0 to 50. 
for i in fake_columns:
    concat1[i] = np.round(np.random.uniform(0.0, 50.0, size=(len(concat1), 1)), 2)

#### Create fake benefit score and ranks.

In [80]:
# Create the fake benefit score based off of Virginia DOT.
concat1["fake_benefit_score"] = ((concat1[fake_columns].sum(axis=1))/ 
                                             (concat1["total_unfunded_need__$1,000"]).apply(pd.to_numeric, errors = 'coerce').fillna(0).sum()
) * 1000000

In [81]:
# Create a fake statewide project rank
concat1["statewide_rank"] = concat1["fake_benefit_score"].rank(
    ascending=False
)

In [82]:
# Create fake project rank by district
concat1["district_rank"] = concat1.groupby("district")[
    "fake_benefit_score"
].rank(method="dense", ascending=False)

In [83]:
# Assign percentile of project among all projects in the state. 
concat1 = _utils.project_size_rating(concat1, 'statewide_rank', 'fake_benefit_score_statewide_percentile')

In [92]:
# Make sure this makes sense
# concat1[['district_rank','statewide_rank','fake_benefit_score','fake_benefit_score_statewide_percentile']].sort_values('fake_benefit_score', ascending= False)

In [85]:
# D4 only projects
len(concat1.loc[concat1["district"] == 4])

120

In [93]:
# list(concat1.columns)

#### Add median across districts
Suggestion from Nick to add medians across the district. 
* Project cost
* unfunded needs
* benefit score

In [87]:
# Grab medians
summary_district_state = (
    concat1.groupby(["district"])
    .agg(
        {
           'total_unfunded_need__$1,000': "median",
            "fake_benefit_score": "median",
            'total_project_cost__$1,000': "median",
           'csis_total_score__out_of_45': "median",
            'atp_total_score__out_of_100': "median"
        }
    )
    .reset_index()
)

In [88]:
# Add suffixes
summary_district_state = summary_district_state.add_suffix("_district_median_")

In [89]:
concat1 = pd.merge(
    concat1,
    summary_district_state,
    left_on="district",
    right_on="district_district_median_",
    how="left",
)

### Save and Clean Up

In [90]:
concat1 = _utils.clean_up_columns(concat1)

In [94]:
# concat1.sample()